In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
from sklearn.metrics import r2_score
import pmdarima as pm

df_veiculo_origem_televisao = pd.read_csv('../df_veiculo_origem_televisao.csv')

df_veiculo_origem_televisao



In [ ]:

df_veiculo_origem_televisao['Ano'] = pd.to_datetime(df_veiculo_origem_televisao['Data']).dt.year
df_veiculo_origem_televisao['Mes'] = pd.to_datetime(df_veiculo_origem_televisao['Data']).dt.month
df_veiculo_origem_televisao = df_veiculo_origem_televisao.drop(columns=[ 'VL Tabela', 'Vl Bruto', 'IPCA BR', 'IPCA ES', 'Taxa Ac. TRI % PIB', 'PMC - Número-índice (2022=100) (Número-índice)/ ES', 'PMC - Número-índice (2022=100) (Número-índice)/ BR', 'PMC - Número-índice com ajuste sazonal (2022=100) (Número-índice)/ ES', 'PMC - Número-índice com ajuste sazonal (2022=100) (Número-índice)/ BR', 'PMC - Variação mês/mês imediatamente anterior, com ajuste sazonal (M/M-1) (%)/ ES', 'PMC - Variação mês/mês imediatamente anterior, com ajuste sazonal (M/M-1) (%)/ BR','Clientes unicos', 'Setor', 'Numero Contratos'])

df_veiculo_origem_televisao

In [ ]:
df_veiculo_origem_televisao = df_veiculo_origem_televisao.groupby(['Data'], as_index=False)['Vl Liquido Final'].sum()

df_veiculo_origem_televisao


In [6]:
target = 'Vl Liquido Final'
X = df_veiculo_origem_televisao.drop(columns=[target])
y = df_veiculo_origem_televisao[target]


In [ ]:
# Convert 'Data' column to datetime
df_veiculo_origem_televisao['Data'] = pd.to_datetime(df_veiculo_origem_televisao['Data'])

# Set 'Data' as the index
df_veiculo_origem_televisao.set_index('Data', inplace=True)

# Define target and features
target = 'Vl Liquido Final'
y = df_veiculo_origem_televisao[target]

# Split the data into train and test sets
train_size = int(len(y) * 0.75)
train, test = y[:train_size], y[train_size:]

# Fit the ARIMA model
model = pm.auto_arima(train, seasonal=True, m=12)

# Make predictions
n_periods = len(test)
predictions = model.predict(n_periods=n_periods)

# Calculate metrics
mse = mean_squared_error(test, predictions)
print(f"Mean Squared Error: {mse}")

r2 = r2_score(test, predictions)
print(f'R^2: {r2}')

mre = mean_absolute_error(test, predictions)
print(f"Mean Absolute Error: {mre}")

# Calculate Root Mean Squared Error (RMSE)
rmse = mean_squared_error(test, predictions, squared=False)
print(f"Root Mean Squared Error: {rmse}")

# Plot the results
plt.figure(figsize=(10, 6))
plt.plot(train.index, train, label='Treino')
plt.plot(test.index, test, label='Teste')
plt.plot(test.index, predictions, label='Previsões', linestyle='--')
plt.xlabel('Data')
plt.ylabel('Vl Liquido Final')
plt.title('Comparação entre Valores Reais e Previsões')
plt.legend()
plt.show()

In [ ]:
feature_importances = model.feature_importances_

features = X.columns
importances_df = pd.DataFrame({'Feature': features, 'Importance': feature_importances})

importances_df = importances_df.sort_values(by='Importance', ascending=False)

plt.figure(figsize=(10, 6))
plt.barh(importances_df['Feature'], importances_df['Importance'])
plt.xlabel('Importância')
plt.ylabel('Feature')
plt.title('Importância das Features')
plt.gca().invert_yaxis()
plt.show()

In [ ]:
max_depth = model.get_depth()
print(f"A profundidade máxima do modelo é: {max_depth}")